<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-28 09:31:57
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.34 C
-------------------
Today PnL: 7.97 K (0.06%)
Current PnL: -16.75 L (-11.67%)
CY Booked + Current PnL: -8.67 L (-6.04%)
-------------------
Total profit:  2.78 L
Total loss:  -19.53 L
-------------------
Total Booked + Current PnL: 18.31 L (15.71%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (6.01%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 74.97 L (55.77%)
Deployed:  1.17 C
Current:  1.34 C
CAGR/XIRR %: 9.25%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-221.55,H-SC,11.59,87682.0,-7634.0,7672.0,0.07,-8.01,8.75,0.04,161.0,-1.00,0.65,72.94,OX40N,NTT,MISC
39,HINDALCO,651.95,761.55,-19.90,H-LC,5.43,110408.0,6096.0,11438.0,-0.49,5.84,10.36,16.81,10.0,0.53,0.82,22.83,X5K,ATH,METALS
4,APOLLOHOSP,6904.43,8285.00,-17.85,H-LC,6.85,111855.0,8289.0,12416.0,-0.15,8.00,11.10,20.00,14.0,0.67,0.83,23.20,X40N,BTT,HEALTHCARE
70,SBIN,760.30,863.00,-11.51,M-LC,4.62,214332.0,12852.0,14360.0,0.28,6.38,6.70,13.51,60.0,0.89,1.60,19.78,XY25,NTT,BANKS
17,BRITANNIA,5190.70,6446.05,12.68,H-LC,9.74,101088.0,7655.0,14941.0,0.39,8.19,14.78,24.18,35.0,0.51,0.75,22.75,XY25,ATH,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,87.81,M-MC,14.10,225014.0,52.0,167613.0,-4.04,0.02,74.49,74.53,193.0,0.00,1.68,30.64,XY24,BTT,STEEL
57,MASFIN,326.60,399.50,-14.63,H-SC,16.88,97095.0,-885.0,22759.0,0.92,-0.90,23.44,22.32,149.0,-0.04,0.72,40.82,XR,ATH,FINANCE
14,BERGEPAINT,561.33,680.00,-19.06,H-MC,11.19,228967.0,1628.0,46435.0,1.32,0.72,20.28,21.14,108.0,0.04,1.70,28.20,XY24,NTT,PAINTS
38,HEROMOTOCO,4311.81,5949.07,-7.14,H-MC,2.81,149513.0,-1400.0,58699.0,0.97,-0.93,39.26,37.97,89.0,-0.02,1.11,24.14,AR,ATH,AUTO
59,NATIONALUM,189.63,247.44,-50.15,H-MC,3.63,100636.0,132.0,30503.0,-1.95,0.13,30.31,30.49,81.0,0.00,0.75,32.58,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.0,-20.46,H-LC,0.96,209350.0,4850.0,36573.0,-0.29,2.37,17.47,20.25,7.0,0.13,1.56,11.31,X40N,BTT,PHARMA
56,LTIM,5564.16,7230.2,-4.50,H-LC,2.42,183888.0,-16422.0,76405.0,0.24,-8.20,41.55,29.94,26.0,-0.21,1.37,27.04,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-7.95,H-LC,2.45,191224.0,-9282.0,58056.0,-0.36,-4.63,30.36,24.33,30.0,-0.16,1.42,10.27,X40N,BTT,FINANCE
40,HINDUNILVR,2413.81,2723.0,-13.47,H-LC,2.96,271141.0,794.0,33838.0,0.23,0.29,12.48,12.81,18.0,0.02,2.02,13.40,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-20.04,H-LC,3.54,158862.0,-42524.0,130140.0,-0.68,-21.12,81.92,43.51,20.0,-0.33,1.18,10.58,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TITAGARH,1097.23,1548.0,-4.51,H-SC,4.19,156468.0,-35547.0,114425.0,2.22,-18.51,73.13,41.08,157.0,-0.31,1.16,32.43,XY24,NTT,ENGINEERING
89,VAIBHAVGBL,350.21,521.0,54.27,H-SC,6.75,102971.0,-49020.0,123143.0,1.88,-32.25,119.59,48.77,126.0,-0.40,0.77,19.02,XR,NTT,JEWELLERY
33,GREENPANEL,375.16,537.0,256.59,M-SC,9.84,159259.0,-23819.0,102802.0,1.83,-13.01,64.55,43.14,234.0,-0.23,1.19,49.80,XY24,NTT,MISC
81,TATAMOTORS,761.77,1065.0,-41.47,M-LC,4.83,254046.0,-22477.0,132561.0,1.81,-8.13,52.18,39.81,54.0,-0.17,1.89,21.75,XY24,NTT,AUTO
9,BAJAJFINSV,1994.80,2758.0,-0.98,H-LC,9.37,98906.0,1161.0,36239.0,1.65,1.19,36.64,38.26,41.0,0.03,0.74,32.07,X40,BTT,FINANCE


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,87.81,M-MC,14.10,225014.0,52.0,167613.0,-4.04,0.02,74.49,74.53,193.0,0.00,1.68,30.64,XY24,BTT,STEEL
59,NATIONALUM,189.63,247.44,-50.15,H-MC,3.63,100636.0,132.0,30503.0,-1.95,0.13,30.31,30.49,81.0,0.00,0.75,32.58,MH,ATH,METALS
25,COLPAL,2714.63,3767.14,-8.42,H-MC,9.52,172840.0,-38901.0,120988.0,-1.93,-18.37,70.00,38.77,83.0,-0.32,1.29,0.00,XY25,ATH,FMCG
77,SURYODAY,189.78,240.00,46.93,H-SC,4.40,103703.0,-46223.0,85897.0,-1.58,-30.83,82.83,26.46,145.0,-0.54,0.77,32.85,XR,NTT,BANKS
55,LAOPALA,369.40,464.00,136.46,H-SC,6.39,75157.0,-25689.0,51513.0,-1.38,-25.47,68.54,25.61,135.0,-0.50,0.56,35.55,AR,NTT,CERAMICS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-221.55,H-SC,11.59,87682.0,-7634.0,7672.0,0.07,-8.01,8.75,0.04,161.0,-1.00,0.65,72.94,OX40N,NTT,MISC
78,SYMPHONY,1306.42,1306.0,-31.08,M-SC,9.77,147034.0,-24107.0,24055.0,0.57,-14.09,16.36,-0.03,194.0,-1.00,1.09,6.40,OX40N,NTT,DURABLES
80,TATAELXSI,7332.28,7332.0,-18.66,H-MC,1.58,85484.0,-17168.0,17165.0,0.73,-16.72,20.08,-0.00,100.0,-1.00,0.64,29.95,OX40N,NTT,IT
86,UJJIVANSFB,52.77,53.0,36.52,M-SC,22.44,118206.0,-24273.0,24894.0,1.06,-17.04,21.06,0.44,249.0,-0.98,0.88,39.52,OX40N,NTT,BANKS
21,CERA,7989.07,8422.0,-16.71,H-SC,3.13,119169.0,-24634.0,32426.0,0.11,-17.13,27.21,5.42,155.0,-0.76,0.89,31.51,OX40N,NTT,CERAMICS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.48,74325.0,39876.0,5091.0,-0.15,115.76,6.85,130.54,1.0,7.83,0.55,13.66,AR,ATH,ELECTRICAL
91,VBL,492.64,672.28,-14.97,H-LC,12.77,258186.0,-6854.0,103507.0,0.58,-2.59,40.09,36.46,2.0,-0.07,1.92,10.26,X40N,ATH,FMCG
82,TCS,3794.03,4998.00,-25.77,H-LC,12.38,241761.0,-54173.0,148079.0,-1.16,-18.31,61.25,31.73,3.0,-0.37,1.80,0.00,X40,BTT,IT
48,INFY,1461.46,2275.00,-16.01,H-LC,6.89,269198.0,7597.0,138018.0,-0.78,2.90,51.27,55.67,5.0,0.06,2.00,9.19,X40,BTT,IT
1,ABB,5551.76,7934.00,-38.50,H-LC,7.22,219434.0,2915.0,89990.0,-0.29,1.35,41.01,42.91,6.0,0.03,1.63,14.70,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,2.44,54693.0,13438.0,3998.0,-0.20,32.57,7.31,42.26,197.0,3.36,0.41,28.77,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,20.18,H-SC,10.44,153990.0,27790.0,18386.0,0.33,22.02,11.94,36.59,154.0,1.51,1.15,37.29,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,2.48,74325.0,39876.0,5091.0,-0.15,115.76,6.85,130.54,1.0,7.83,0.55,13.66,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,87.81,M-MC,14.10,225014.0,52.0,167613.0,-4.04,0.02,74.49,74.53,193.0,0.00,1.68,30.64,XY24,BTT,STEEL
53,KPIGREEN,497.21,731.64,27.50,H-SC,12.80,130536.0,5239.0,53833.0,-0.36,4.18,41.24,47.15,143.0,0.10,0.97,63.31,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,20.18,H-SC,10.44,153990.0,27790.0,18386.0,0.33,22.02,11.94,36.59,154.0,1.51,1.15,37.29,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,-0.58,H-SC,1.20,111625.0,9621.0,34727.0,-1.18,9.43,31.11,43.48,122.0,0.28,0.83,27.79,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2326.38,-6.19,H-SC,10.49,175980.0,11310.0,56666.0,0.72,6.87,32.20,41.28,119.0,0.20,1.31,15.40,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.48,74325.0,39876.0,5091.0,-0.15,115.76,6.85,130.54,1.0,7.83,0.55,13.66,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1219.98,2270.00,-38.43,M-SC,8.84,103620.0,12122.0,66628.0,-1.24,13.25,64.30,86.07,205.0,0.18,0.77,49.90,XR,NTT,DURABLES
45,INDIAMART,2327.09,4850.92,-47.33,H-SC,12.89,136878.0,13542.0,120220.0,0.86,10.98,87.83,108.45,118.0,0.11,1.02,35.79,AR,ATH,MISC
53,KPIGREEN,497.21,731.64,27.50,H-SC,12.80,130536.0,5239.0,53833.0,-0.36,4.18,41.24,47.15,143.0,0.10,0.97,63.31,MH,ATH,POWER
95,WIPRO,243.46,420.00,-8.76,M-LC,5.66,156550.0,5605.0,103855.0,-0.67,3.71,66.34,72.51,56.0,0.05,1.17,9.96,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,5.75,20336.0,-29257.0,458.0,-0.89,-58.99,2.25,-58.07,204.0,-63.88,0.15,36.38,XY24,ATH,MISC
28,DIGITIDE,188.38,267.99,NaN,NaN,2.44,54693.0,13438.0,3998.0,-0.20,32.57,7.31,42.26,197.0,3.36,0.41,28.77,XY24,ATH,IT
74,SIS,477.00,477.00,2310.36,M-SC,5.81,64855.0,-16235.0,16233.0,0.38,-20.02,25.03,0.00,238.0,-1.00,0.48,31.44,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,21.03,69331.0,-44218.0,84272.0,0.17,-38.94,121.55,35.27,267.0,-0.52,0.52,69.69,XR,NTT,HOTELS
84,TRIDENT,37.35,48.00,24.75,M-SC,4.71,78373.0,-13882.0,40190.0,0.76,-15.05,51.28,28.51,236.0,-0.35,0.58,34.11,XR,NTT,TEXTILES


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,2.44,54693.0,13438.0,3998.0,-0.20,32.57,7.31,42.26,197.0,3.36,0.41,28.77,XY24,ATH,IT
16,BLUSPRING,226.45,94.95,NaN,NaN,5.75,20336.0,-29257.0,458.0,-0.89,-58.99,2.25,-58.07,204.0,-63.88,0.15,36.38,XY24,ATH,MISC
33,GREENPANEL,375.16,537.00,256.59,M-SC,9.84,159259.0,-23819.0,102802.0,1.83,-13.01,64.55,43.14,234.0,-0.23,1.19,49.80,XY24,NTT,MISC
58,MEDANTA,1087.93,1486.00,20.18,H-SC,10.44,153990.0,27790.0,18386.0,0.33,22.02,11.94,36.59,154.0,1.51,1.15,37.29,XY24,NTT,HEALTHCARE
72,SHALBY,261.39,327.00,971.77,M-SC,9.74,145842.0,-35301.0,80767.0,1.40,-19.49,55.38,25.10,228.0,-0.44,1.09,16.88,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.43
1,25,41.07
2,50,70.04


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.82
LC    35.01
MC    22.05
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.99
X40N     14.52
X40      12.47
XY25     10.75
XR        9.20
OX40N     8.34
AR        8.24
X5K       2.24
MH        1.72
X200      1.37
SR        1.15
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.71
H-SC    25.38
H-MC    19.02
M-SC    15.02
M-LC     7.24
M-MC     2.68
L-SC     1.42
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.02,-3.84,38.58
IT,11.77,-15.01,71.68
FINANCE,8.38,-15.87,57.91
BANKS,7.60,-11.66,57.23
MISC,6.11,-17.23,71.97
PAINTS,5.56,-16.68,39.68
ELECTRICAL,5.41,-1.28,49.27
HEALTHCARE,4.62,2.67,23.17
AUTO,4.42,-13.80,63.31


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2689502.0,25
XR,1058848.0,13
AR,1037869.0,10
X40,777922.0,10
X40N,597929.0,12
XY25,495799.0,8
OX40N,351007.0,11
SR,188770.0,2
X5K,138366.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2410673.0,26
M-SC,1613190.0,20
H-LC,1277027.0,20
H-MC,1148335.0,15
M-LC,407557.0,5
M-MC,317842.0,2
L-SC,209375.0,3
L-MC,55668.0,1
L-LC,47539.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,936138.0,7
M-SC,XY24,770189.0,7
H-SC,AR,515808.0,3
H-LC,X40,505488.0,6
H-MC,XY24,444630.0,4
H-SC,XR,350901.0,4
M-MC,XY24,317842.0,2
H-LC,X40N,313029.0,6
M-SC,XR,283567.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
